In [24]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

symbols = [ "H","Be", "H"]
coordinates = np.array([ 0.0,0.0,-1.29,0.0,0.0,0.0, 0.0,0.0,1.29])
basis_set = "sto-3g"


H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    active_electrons=4,
    active_orbitals=6
)

In [25]:
singles, doubles = qchem.excitations(electrons=4, orbitals=qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 92


In [26]:
hf = qchem.hf_state(4, qubits)

In [27]:


num_theta = len(singles) + len(doubles)

def circuit_VQE(params,expectations ):
    qml.AllSinglesDoubles(
        weights = params,
        wires = range(qubits),
        hf_state = hf,
        singles = expectations[0],
        doubles = expectations[1])
    return qml.expval(H)

In [28]:
dev = qml.device("default.qubit", wires=qubits)
cost_fn = qml.QNode(circuit_VQE, dev)

circuit_gradient = qml.grad(cost_fn, argnum=0)
n=len(doubles)+ len(singles)
params = [0.0] * n
expectations = [singles,doubles]
grads = circuit_gradient(params, expectations)

for i in range(n):
    print(f"Gradient: {grads[i]}")

Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: -1.7863965862119358e-10
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 1.7863960311004234e-10
Gradient: 0.0
Gradient: 0.0
Gradient: 3.0811462409241486e-10
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: -3.081145061312185e-10
Gradient: 0.0
Gradient: -0.020670475965201416
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: -0.02067047596520534
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: -0.10182127165190102
Gradient: 0.0
Gradient: 0.0
Gradient: -0.08210828451383691
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: -0.005001773293946419
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: -0.0653745040480538
Gradient: 0.07037627734200014
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradient: 0.0
Gradie

In [29]:
single_select = [singles[i] for i in range(len(singles)) if abs(grads[i])>2e-10]

In [30]:
n_s = len(singles)
double_select = [doubles[i] for i in range(len(doubles)) if abs(grads[n_s+i])>1e-4]

In [31]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time
cost_fn = qml.QNode(circuit_VQE, dev)
stepsize = 0.4
opt = qml.GradientDescentOptimizer(stepsize=stepsize)
params = np.zeros(len(double_select + single_select), requires_grad=True)

gates_select =  [single_select,double_select]

for n in range(20):
    t1 = time.time()
    params, energy = opt.step_and_cost(cost_fn, params, expectations=gates_select)
    t2 = time.time()
    print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))

n = 0,  E = -14.77344237 H, t = 1.94 s
n = 1,  E = -14.78991989 H, t = 2.10 s
n = 2,  E = -14.79582241 H, t = 2.00 s
n = 3,  E = -14.79863957 H, t = 2.10 s
n = 4,  E = -14.80020550 H, t = 2.10 s
n = 5,  E = -14.80116394 H, t = 2.10 s
n = 6,  E = -14.80178956 H, t = 2.00 s
n = 7,  E = -14.80221576 H, t = 2.10 s
n = 8,  E = -14.80251435 H, t = 1.80 s
n = 9,  E = -14.80272740 H, t = 2.10 s
n = 10,  E = -14.80288132 H, t = 2.10 s
n = 11,  E = -14.80299347 H, t = 2.20 s
n = 12,  E = -14.80307573 H, t = 2.00 s
n = 13,  E = -14.80313637 H, t = 2.10 s
n = 14,  E = -14.80318125 H, t = 2.10 s
n = 15,  E = -14.80321459 H, t = 2.20 s
n = 16,  E = -14.80323942 H, t = 1.90 s
n = 17,  E = -14.80325798 H, t = 2.10 s
n = 18,  E = -14.80327187 H, t = 2.00 s
n = 19,  E = -14.80328229 H, t = 2.10 s


In [3]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
# pylint: enable=line-too-long

import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit_nature.settings import settings
settings.dict_aux_operators = False
#from  qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver

/tmp/ipykernel_557/4124333501.py:2: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms import (GroundStateEigensolver,
/tmp/ipykernel_557/4124333501.py:2: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms import (GroundStateEigensolver,
/opt/conda/envs/pennylane/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3553: ListAuxOpsDeprecationWarning: List-based `aux_operators` are deprecated as of version 0.3.0 and support for them will be removed no sooner than 3 mo

In [11]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["Be", [0.0, 0.0, 0.0]],
            ["H", [-dist, 0.0, 0.0]],
            ["H", [dist, 0.0, 0.0]]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    

    driver = ElectronicStructureMoleculeDriver(
         molecule=molecule,
         basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)
    
    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [FreezeCoreTransformer(freeze_core=True,
                               remove_orbitals=[-2,-1])])

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles
    print(num_spin_orbitals,num_particles)
    mapper = ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops[0]  # Set Hamiltonian
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [12]:
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

backend = BasicAer.get_backend("statevector_simulator")
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

# pylint: disable=undefined-loop-variable


dist = 1.291
(qubit_op, num_particles, num_spin_orbitals,
                            problem, converter) = get_qubit_op(dist)
result = exact_solver(problem,converter)
exact_energies.append(result.total_energies[0].real)
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
var_form = UCCSD(converter,
                    num_particles,
                    num_spin_orbitals,
                    initial_state=init_state)
vqe = VQE(var_form, optimizer, quantum_instance=backend)
print('1')
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
print('2')
vqe_result = problem.interpret(vqe_calc).total_energies[0].real
vqe_energies.append(vqe_result)
print(f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Exact Energy: {exact_energies[-1]:.5f}")

print("All energies have been calculated")

12 (2, 2)


/tmp/ipykernel_326/885534758.py:3: NatureDeprecationWarning: The qiskit_nature.algorithms.ground_state_solvers.GroundStateEigensolver class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.ground_state_solvers.GroundStateEigensolver class.
  calc = GroundStateEigensolver(converter, solver)


1
2
Interatomic Distance: 1.29 VQE Result: -15.59408 Exact Energy: -15.59443
All energies have been calculated
